In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
import random
import numpy as np
import math
from sklearn import tree
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
def indexOf(d,v): # d là một dictionary, v là một feature.
  for i in d:
    if (i == v):
        return True
  return False

In [ ]:
def get_feature(tree):
  features = [i for i in tree.tree_.feature]
  featureIndex = [num for num in features if num != -2]
  return featureIndex

In [ ]:
def get_featureFirst(d,n):
  number = n if n < len(d) else len(d)
  count = 0
  arr = []
  for key, v in d:
    if (count < number):
      arr.append(key)
      count = count + 1
    else:
      break
  return arr


In [ ]:
data = pd.read_csv("/content/drive/MyDrive/data/datafs/colon1.csv")
X = data.iloc[:,1:]
y = data.iloc[:,0]
d = {}
for i in range(50):
  X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.7,random_state=np.random.randint(0,10000), stratify = y)
  rf_model = RandomForestClassifier() # default create 100 trees
  rf_model.fit(X_train, y_train)
  for idx, dtree in enumerate(rf_model.estimators_):
      a = get_feature(tree = dtree)
      for i in a: # a là một list features trong model.
        if(indexOf(d, i)):
          number = d.get(i)
          number = number + 1
          d[i] = number
        else:
          d.update({i:1})

In [ ]:
X_original = data.iloc[:,1:]
y = data.iloc[:,0]
c_original = X_original.shape[1]
number_feat_selected = round(c_original * 1/100)

In [ ]:
number = round(c_original * 5/100)
a = sorted(d.items(), key = lambda item: item[1], reverse = True)
index_5_percent = get_featureFirst(a,number) # top 5% features
top_5_percent_feature_names = X.columns[index_5_percent]
X_new = X_original.iloc[:,index_5_percent] # dataframe use 5% features


In [ ]:
list_values = {}
list_values.update({
    "X": X,
    "y": y,
    "d": d,
    "number_feat_selected": number_feat_selected, # top 1%
    "top_5_percent_feature_names": top_5_percent_feature_names,
    "X_new": X_new
})

In [ ]:
import pickle
import time
with open('/content/drive/My Drive/data/list_values3.pkl', 'wb') as file:
  pickle.dump(list_values, file)

list_times = {}
with open('/content/drive/My Drive/data/list_times3.pkl', 'wb') as file:
  pickle.dump(list_times, file)

# **Filter**

In [ ]:
import pickle
import time
import pandas as pd
with open('/content/drive/My Drive/data/list_values3.pkl', 'rb') as file:
  list_values = pickle.load(file)

with open('/content/drive/My Drive/data/list_times3.pkl', 'rb') as file:
  list_times = pickle.load(file)

# Mutual Information
from sklearn.feature_selection import mutual_info_classif as mic
start_time = time.time()
importances = mic(list_values["X_new"],list_values["y"])
IG_feature_names = pd.Series(importances, index = list_values["X_new"].columns).sort_values(ascending = False).head(list_values["number_feat_selected"]).index #ascending  = False nghĩa là sẽ sắp xếp các giá trị nhỏ dần, lấy tên features
end_time = time.time()
elapsed_time = round(end_time - start_time, 3)

list_values["IG_feature_names"] = IG_feature_names
with open('/content/drive/My Drive/data/list_values3.pkl', 'wb') as file:
  pickle.dump(list_values, file)

list_times["IG_time"] = elapsed_time
with open('/content/drive/My Drive/data/list_times3.pkl', 'wb') as file:
  pickle.dump(list_times, file)

In [ ]:
pip install skfeature-chappers

In [ ]:

import pickle
import time
import pandas as pd
with open('/content/drive/My Drive/data/list_values3.pkl', 'rb') as file:
  list_values = pickle.load(file)

with open('/content/drive/My Drive/data/list_times3.pkl', 'rb') as file:
  list_times = pickle.load(file)

#Fisher's score
from skfeature.function.similarity_based import fisher_score
X_new_FS = list_values["X_new"].to_numpy()
y_FS = list_values["y"].to_numpy()

start_time = time.time()
ranks = fisher_score.fisher_score(X_new_FS, y_FS)
Fish_feature_names = pd.Series(ranks, index = list_values["X_new"].columns).sort_values(ascending = False).head(list_values["number_feat_selected"]).index
end_time = time.time()
elapsed_time = round(end_time - start_time, 3)

list_values["Fish_feature_names"] = Fish_feature_names
with open('/content/drive/My Drive/data/list_values3.pkl', 'wb') as file:
  pickle.dump(list_values, file)

list_times["Fish_time"] = elapsed_time
with open('/content/drive/My Drive/data/list_times3.pkl', 'wb') as file:
  pickle.dump(list_times, file)

In [ ]:
import pickle
import pandas as pd
import time
with open('/content/drive/My Drive/data/list_values3.pkl', 'rb') as file:
  list_values = pickle.load(file)

with open('/content/drive/My Drive/data/list_times3.pkl', 'rb') as file:
  list_times = pickle.load(file)

#Pearson Correlation
X_new_corr = list_values["X_new"].copy()
X_new_corr['class'] = list_values["y"]
start_time = time.time()
corr = X_new_corr.corr()
high_corr_features = corr['class'].abs().drop('class') # xóa tương quan của class
Corr_feature_names = pd.Series(high_corr_features, index = list_values["X_new"].columns).sort_values(ascending = False).head(list_values["number_feat_selected"]).index
end_time = time.time()
elapsed_time = round(end_time - start_time, 3)

list_values["Corr_feature_names"] = Corr_feature_names
with open('/content/drive/My Drive/data/list_values3.pkl', 'wb') as file:
  pickle.dump(list_values, file)

list_times["Corr_time"] = elapsed_time
with open('/content/drive/My Drive/data/list_times3.pkl', 'wb') as file:
  pickle.dump(list_times, file)

# **WRAPPER**

In [ ]:
import pickle
import time
with open('/content/drive/My Drive/data/list_values3.pkl', 'rb') as file:
  list_values = pickle.load(file)

with open('/content/drive/My Drive/data/list_times3.pkl', 'rb') as file:
  list_times = pickle.load(file)
#forward
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LogisticRegression
start_time = time.time()
lr = LogisticRegression()
ffs = SequentialFeatureSelector(lr, k_features= list_values["number_feat_selected"], forward = True, n_jobs = -1) # n_jobs = -1: chỉ định số jobs được thực hiện cùng lúc.
ffs.fit(list_values["X_new"],list_values["y"])
For_feature_names= list(ffs.k_feature_names_)
end_time = time.time()
elapsed_time = round(end_time - start_time, 3)

list_values["For_feature_names"] = For_feature_names
with open('/content/drive/My Drive/data/list_values3.pkl', 'wb') as file:
  pickle.dump(list_values, file)

list_times["For_time"] = elapsed_time
with open('/content/drive/My Drive/data/list_times3.pkl', 'wb') as file:
  pickle.dump(list_times, file)

In [ ]:
import pickle
import time
with open('/content/drive/My Drive/data/list_values3.pkl', 'rb') as file:
  list_values = pickle.load(file)

with open('/content/drive/My Drive/data/list_times3.pkl', 'rb') as file:
  list_times = pickle.load(file)
#backward
from sklearn.linear_model import LogisticRegression
from mlxtend.feature_selection import SequentialFeatureSelector
start_time = time.time()
lr = LogisticRegression()
lr.fit(list_values["X_new"],list_values["y"])
bfs = SequentialFeatureSelector(lr, k_features = list_values["number_feat_selected"], forward = False, n_jobs = -1)
bfs.fit(list_values["X_new"],list_values["y"])
Back_feature_names = bfs.k_feature_names_
end_time = time.time()
elapsed_time = round(end_time - start_time, 3)

list_values["Back_feature_names"] = Back_feature_names
with open('/content/drive/My Drive/data/list_values3.pkl', 'wb') as file:
  pickle.dump(list_values, file)

list_times["Back_time"] = elapsed_time
with open('/content/drive/My Drive/data/list_times3.pkl', 'wb') as file:
  pickle.dump(list_times, file)

In [ ]:
import pickle
import time
with open('/content/drive/My Drive/data/list_values3.pkl', 'rb') as file:
  list_values = pickle.load(file)

with open('/content/drive/My Drive/data/list_times3.pkl', 'rb') as file:
  list_times = pickle.load(file)
#Recursive feature elimmination
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
start_time = time.time()
lr = LogisticRegression()
rfe = RFE(lr, n_features_to_select = list_values["number_feat_selected"])
rfe.fit(list_values["X_new"],list_values["y"])
Rec_feature_names = list_values["X_new"].columns[rfe.support_].tolist()
end_time = time.time()
elapsed_time = round(end_time - start_time, 3)

list_values["Rec_feature_names"] = Rec_feature_names
with open('/content/drive/My Drive/data/list_values3.pkl', 'wb') as file:
  pickle.dump(list_values, file)

list_times["Rec_time"] = elapsed_time
with open('/content/drive/My Drive/data/list_times3.pkl', 'wb') as file:
  pickle.dump(list_times, file)

# **EMBEDDED**

In [ ]:
import pickle
import time
import pandas as pd
import numpy as np
with open('/content/drive/My Drive/data/list_values3.pkl', 'rb') as file:
  list_values = pickle.load(file)

with open('/content/drive/My Drive/data/list_times3.pkl', 'rb') as file:
  list_times = pickle.load(file)
#LASSO
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
start_time = time.time()
logistic = LogisticRegression(penalty = 'l1', solver = 'liblinear', random_state = np.random.randint(0,10000)).fit(list_values["X_new"],list_values["y"]) # l1: là chỉ ra sẽ sử dụng lasso, solver = 'liblinear': được sử dụng để model tìm các hệ số tối ưu.
model = SelectFromModel(logistic, prefit = True) # prefit = True nghĩa là đã huấn luyện model từ trước, nên khi cho vào chỉ việc giảm feature.
X_new_lasso = model.transform(list_values["X_new"])
selected_features = list_values["X_new"].columns[model.get_support()] # gets the names of the selected features
coefficients = logistic.coef_[0][model.get_support()] # gets the coefficients of the selected features
features_coefficients = pd.DataFrame({'feature': selected_features, 'coefficient': coefficients})
features_coefficients = features_coefficients.reindex(features_coefficients.coefficient.abs().sort_values(ascending=False).index)
top_features = features_coefficients.head(list_values["number_feat_selected"])
Lasso_feature_names = top_features['feature'].values
end_time = time.time()
elapsed_time = round(end_time - start_time, 3)

list_values["Lasso_feature_names"] = Lasso_feature_names
with open('/content/drive/My Drive/data/list_values3.pkl', 'wb') as file:
  pickle.dump(list_values, file)

list_times["Lasso_time"] = elapsed_time
with open('/content/drive/My Drive/data/list_times3.pkl', 'wb') as file:
  pickle.dump(list_times, file)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import pandas as pd
import pickle

with open("/content/drive/My Drive/data/list_values3.pkl", "rb") as file:
  list_values = pickle.load(file)



classifiers = {
    'Decision Tree': DecisionTreeClassifier(),
    'Naive Bayes': GaussianNB(),
    'SVC': SVC(),
    'KNN': KNeighborsClassifier()
}


feature_methods = {
'all': None,  # Using all features
'5%': list_values["top_5_percent_feature_names"] ,
'IG': list_values["IG_feature_names"] ,
'CHI': list_values["Chi2_feature_names"] ,
'FISH': list_values["Fish_feature_names"] ,
'CORR': list_values["Corr_feature_names"] ,
'VAR': list_values["Var_feature_names"] ,
'MAD': list_values["MAD_feature_names"]  ,
'DIS': list_values["Dis_feature_names"] ,
'FOR': list_values["For_feature_names"] ,
'BACK': list(list_values["Back_feature_names"]) ,
'RECURSIVE': list_values["Rec_feature_names"] ,
'LASSO': list_values["Lasso_feature_names"]
}



def evaluate_model(clf, X_train, y_train, X_test, y_test):
    clf.fit(X_train, y_train.values.ravel())
    return accuracy_score(y_test, clf.predict(X_test))


accuracies = {method: {clf: [] for clf in classifiers} for method in feature_methods}


for i in range(50):
    X_train, X_test, y_train, y_test = train_test_split(list_values["X"] , list_values["y"] , train_size=0.7, random_state=np.random.randint(0, 10000), stratify = list_values["y"]) # X,y là từ data ban đầu

    for method, feature_names in feature_methods.items():
      if method == 'all':
        X_train_method =  X_train
        X_test_method  =  X_test
      else:
        X_train_method = X_train[feature_names]
        X_test_method = X_test[feature_names]
      for clf_name, clf in classifiers.items():
        acc = evaluate_model(clf, X_train_method, y_train, X_test_method, y_test)
        accuracies[method][clf_name].append(acc)


for method in feature_methods:
    mean_results = {'Algorithm': list(classifiers.keys()), 'Mean Accuracy': [np.mean(accuracies[method][clf]) for clf in classifiers]}
    mean_results_df = pd.DataFrame(mean_results)

    detailed_results = {f'{clf} Accuracies': accuracies[method][clf] for clf in classifiers}
    detailed_results_df = pd.DataFrame(detailed_results)
    if method == 'all':
      filename = f'model_accuracies_All_Feature.xlsx'
    elif method == '5%':
      filename = f'model_accuracies_5%_Feature.xlsx'
    else:
      filename = f'model_accuracies_1%_{method}_Feature.xlsx'
    with pd.ExcelWriter(filename) as writer:
        mean_results_df.to_excel(writer, sheet_name='Mean Accuracies', index=False)
        detailed_results_df.to_excel(writer, sheet_name='Detailed Accuracies', index=False)

    print(f"Kết quả đã được ghi vào file {filename}")